In [1]:
#r "nuget:OpenCvSharp4.Windows, 4.2.0.20200208"
#r "nuget: AForge.Imaging, 2.2.5"

using OpenCvSharp;
using AForge.Imaging;

Installing package OpenCvSharp4.Windows, version 4.2.0.20200208.......

Installed package AForge.Imaging version 2.2.5

Unhandled exception: input.fsx (2,7)-(2,18) typecheck error The value or constructor 'OpenCvSharp' is not defined.
input.fsx (3,7)-(3,13) typecheck error The value, namespace, type or module 'AForge' is not defined.

In [2]:
using System;
using System.Drawing;
using System.IO;
using System.Linq;

In [3]:
var imagesFolderPath = "F:/Palm-vein-ID/small-db"; // path to images

In [4]:
var imagesOutputDirectory = "F:/Palm-vein-ID/small-db/output"; // path to output directory

In [5]:
public class PalmModel {
    public string FileName { get; set; }
    public string Directory { get; set; }
    public OpenCvSharp.Mat SourceImage { get; set; }
    public OpenCvSharp.Mat ThresholdImage { get; set; } // = binary
    
    // get image params
    public int Width { get; set; }
    public int Height { get; set; }
    
    public OpenCvSharp.Mat ROI { get; set; }
}

public List<PalmModel> listOfPalms = new List<PalmModel>();

In [6]:
var binaryROIFolder = "F:/Palm-vein-ID/small-db/output/bin/";

In [7]:
// get list of all images
var listOfImages = Directory.GetFiles(imagesFolderPath, "*.jpg")
    .ToList();
    
// get list of all filenames without .jpg
listOfImages.ForEach(x => {
        x = x.Remove(0, x.LastIndexOf('\\') + 1)
            .Replace(".jpg", "");
        
        listOfPalms.Add(new PalmModel() {
            FileName = x
        });
    }
);
                                               
Console.WriteLine($"Total palm collection: {listOfPalms.Count}");
Console.WriteLine(listOfPalms[0].FileName);

display(
    span(
        a[href: "#"](b("Sample source image")),
        img[src: "sample/source.jpg", height: 400, width: 400] /* used random image */
    )
);

Total palm collection: 6
001_l_460_01


Sample source image

In [8]:
// create directories for 
listOfPalms.ForEach(x => {
    x.Directory = $"{imagesOutputDirectory}\\{x.FileName}";
    
    // remove directory if not empty
    if(Directory.Exists(x.Directory) 
       && Directory.GetFiles(x.Directory).Length > 0) {
        Directory.Delete(x.Directory, true); // recursive
    }
    Directory.CreateDirectory(x.Directory);
});

In [9]:
// add 7200 images to collection
listOfPalms.ForEach(x => {
    var path = imagesFolderPath + "\\" + x.FileName + ".jpg";
    x.SourceImage = OpenCvSharp.Cv2.ImRead(
        path, OpenCvSharp.ImreadModes.Grayscale);
    
    if(!File.Exists($"{x.Directory}\\source.jpg"))
        File.Copy(path, $"{x.Directory}\\source.jpg");
    
    x.Height = x.SourceImage.Size().Height;
    x.Width = x.SourceImage.Size().Width;
});

// debug
Console.WriteLine(listOfPalms[0].SourceImage);
Console.WriteLine($"Height: {listOfPalms[0].Height}; Width: {listOfPalms[0].Width}");

Mat [ 576*768*CV_8UC1, IsContinuous=True, IsSubmatrix=False, Ptr=0x2e079231880, Data=0x2e4d35e6140 ]
Height: 576; Width: 768


In [10]:
// apply threshold
listOfPalms.ForEach(x => {
    x.ThresholdImage = x.SourceImage.Threshold (0, 255, 
                                    OpenCvSharp.ThresholdTypes.Otsu);
    
    // save for debug
    OpenCvSharp.Cv2.ImWrite(x.Directory + "\\binary.jpg", x.ThresholdImage);
});

In [11]:
display(
    span(
        a[href: "#"](b("Binary image sample")),
        img[src: "sample/binary.jpg", height: 400, width: 400] /* used random image */
    )
);

display(HTML("<center><b>ROI extraction</b></center><hr />"));

Binary image sample

ROI extraction

In [12]:
listOfPalms.ForEach(x => {
    var i1 = x.Height - 50;
    var i2 = x.Width - 50;

    var radius = 50;
    int pX = 0;
    int pY = 0;

    for(int i = 50; i != i1; i++) {
        for(int j = 50; j != i2; j++) {
            if(x.ThresholdImage.Get<byte>(i, j) == 255) {
                int a = 0;
                for (a = 1; a < 360; a++) {
                    var y1 = Convert.ToInt16 (j + radius * Math.Cos (a * Math.PI / 180));
                    var x1 = Convert.ToInt16 (i - radius * Math.Sin (a * Math.PI / 180));

                    if (x1 < 1 || x1 > i1 || y1 < 1 || y1 > i2 || x.ThresholdImage.Get<byte> (x1, y1) == 0)
                        break;
                    }

                if (a == 360) {
                    radius += 10;
                    pX = i;
                    pY = j;
                }
            }
        }
    }

    radius = radius - 10;

    var x0 = Convert.ToInt16 (pY - Math.Sqrt (2) * radius / 2);
    var y0 = Convert.ToInt16 (pX - Math.Sqrt (2) * radius / 2);
    var wsize = Convert.ToInt16 (Math.Sqrt (2) * radius);

    var rect = new OpenCvSharp.Rect(x0, y0, wsize, wsize);
    
    // for visual debug
    OpenCvSharp.Mat drawROIImage = new OpenCvSharp.Mat();
    x.SourceImage.CopyTo(drawROIImage);
    drawROIImage.Rectangle(rect, OpenCvSharp.Scalar.White);
    
    x.ROI = new OpenCvSharp.Mat(x.SourceImage, rect)
        .Resize(new OpenCvSharp.Size(216, 216));   
    OpenCvSharp.Cv2.ImWrite(x.Directory + "\\ROIOnSource.jpg", drawROIImage);
    OpenCvSharp.Cv2.ImWrite(x.Directory + "\\ROI.jpg", x.ROI);
});

In [13]:
display(
    span(
        a[href: "#"](b("Extracted ROI on source image")),
        img[src: "sample/ROIOnSource.jpg", height: 400, width: 400] /* used random image */
    )
);

Extracted ROI on source image

In [14]:
display(
    span(
        a[href: "#"](b("ROI")),
        img[src: "sample/ROI.jpg"] /* used random image */
    )
);

ROI

In [15]:
Console.WriteLine($"[{DateTime.Now}] Creating dump file with ROI images. ");

listOfPalms.ForEach(x => {
    using (var fs = new OpenCvSharp.FileStorage($"{binaryROIFolder}{x.FileName}.bin", 
                                    OpenCvSharp.FileStorage.Mode.Write | OpenCvSharp.FileStorage.Mode.FormatYaml))
    {
        fs.Write("FileName", x.FileName);
        fs.Write("Directory", x.Directory);
        fs.Write("SourceImage", x.SourceImage);
        fs.Write("ThresholdImage", x.ThresholdImage);
        fs.Write("Width", x.Height);
        fs.Write("Height", x.Height);
        fs.Write("ROI", x.ROI);
    }
});

Console.WriteLine($"[{DateTime.Now}] Dump created! {binaryROIFolder}");

[05.03.2020 21:12:39] Creating dump file with ROI images. 
[05.03.2020 21:12:40] Dump created! F:/Palm-vein-ID/small-db/output/bin/
